참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import matplotlib.patches as patch # 도형 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 10.002b

p. 435

보의 처짐과 굽힘 모멘트 사이의 관계는 다음과 같다.(p. 236 eq. 6.3)<br>
Following is the relationship between the deflection of a beam and the bending moment.(p. 236 eq. 6.3)

$$
\frac{d^2\nu}{dx^2}=\frac{M}{EI}
$$

처짐 $\nu$와 축방향 하중 $P$에 의한 굽힘모멘트 $M$은 다음과 같다.<br>The bending moment $M$ due to deflection $\nu$ and axial load $P$ is as follows.

$$
M = -P \nu
$$

$$
\frac{d^2\nu}{dx^2}=\frac{M}{EI}=\frac{-P}{EI}\nu
$$

$$
\frac{d^2\nu}{dx^2}+\frac{P}{EI}\nu=0
$$

### 일반해<br>General Solution

아래와 같은 함수가 위 미분 방정식을 만족하는지 알아보자.<br>Let's see if following function satisfies the equation above.

$$
\nu(x)=C_1 sin\left( \sqrt{ \frac{P}{EI}} x \right) + C_2 cos\left(\sqrt{ \frac{P}{EI}} x \right)
$$

In [ ]:
P, E, I, L = sy.symbols('P, E, I, L', positive=True , nonzero=True)
x = sy.symbols('x', nonnegative=True)
C1, C2 = sy.symbols('C1, C2', real=True)
P, E, I, x, C1, C2 = sy.symbols('P, E, I, x, C1, C2')
w = sy.sqrt(P / (E*I))
nu = C1 * sy.sin(w * x) + C2 * sy.cos(w * x)

In [ ]:
nu

두번 미분<br>Differentiate twice

In [ ]:
nu.diff(x)

In [ ]:
nu.diff(x, 2)

방정식에 대입<br>Substitute back to the equation

In [ ]:
nu.diff(x, 2) + P / (E * I) * nu

### 적분 상수 결정<br>Decide integration constants

경계조건으로 적분 상수를 결정한다.<br>Using boundary conditions, decide integration constants.

$$
\nu(x)=C_1 sin\left( \sqrt{ \frac{P}{EI}} x \right) + C_2 cos\left(\sqrt{ \frac{P}{EI}} x \right)
$$

#### 단순지지보<br>Simply supported

$x=0$

$$
\nu(0)=C_2 cos\left(\sqrt{ \frac{P}{EI}} x \right)
$$

$$
C_2 = 0
$$

$x=L$

$$
\nu(L)=C_1 sin\left( \sqrt{ \frac{P}{EI}} L \right)=0
$$

$$
\sqrt{ \frac{P}{EI}} L =n \pi
$$

$$
\frac{PL^2}{EI}=n^2 \pi^2
$$

$$
P_{cr}=\frac{n^2 \pi^2 EI}{L^2}
$$

여기서 $n$이 자연수 일 때 의미 있는 해가 얻어진다.<br>$n \in \mathbb{N}$ gives a meaningful solution.

$C_1$ 값은 결정되지 않는데 변위의 크기가 매우 클 수도 있다고 볼 수 있다.<br>$C_1$ is undecided; the deflection might be very large.

In [ ]:
eq_boundary = sy.Eq(C1 * sy.sin(L * sy.sqrt(P / (E * I))))

In [ ]:
eq_boundary

In [ ]:
sy.solve(eq_boundary, P)

In [ ]:
sy.solve(eq_boundary, C1)

##### 모드 형상<br>Mode shapes

In [ ]:
plt.clf()
y_array = np.arange(0, 1, 1e-3)
for n in range(1, 3 + 1):
    x_array = 0.1 * np.sin((np.pi) * n * y_array)
    plt.plot(x_array, y_array, label='n=%d'%n)
plt.ylabel('y/L')
plt.axis('equal')
plt.legend(loc=0)
plt.grid(True)
plt.show()

#### 다른 경계 조건<br>Other boundary conditions

|          |          |       $L_e$         |       $P_{cr}$      |    |
|:--------:|:--------:|:-------------------:|:-------------------:|:-----:|
| 단순지지<br>Simply supported | 단순지지<br>Simply supported |       $L$           | $\frac{\pi ^ 2 EI}{L^2}$ | fig 10.2, 10.3, 10.4 |
|  고정단<br>Fixed  |  고정단<br>Fixed  |      $\frac{1}{2}L$ | $4\frac{\pi ^ 2 EI}{L^2}$ | fig. 10.5 |
|  고정단<br>Fixed  |  자유단<br>Free  |       $2L$          | $\frac{1}{4}\frac{\pi ^ 2 EI}{L^2}$ | fig. 10.6 |
|  고정단<br>Fixed  | 단순지지<br>Simply supported |       $0.7 L$       | $2\frac{\pi ^ 2 EI}{L^2}$ | fig. 10. 7 |


In [ ]:
plt.clf()
y_array = np.arange(0, 1, 1e-3)

x_array = 0.1 * np.sin((np.pi) * y_array)
plt.plot(x_array, y_array, label='simple-simple')

x_array = 0.1 * np.cos((2 * np.pi) * y_array) - 0.1
plt.plot(x_array, y_array, label='fix-fix')

x_array = 0.1 * np.cos((0.5 * np.pi) * y_array) - 0.1
plt.plot(x_array, y_array, label='fix-free')

plt.ylabel('y/L')
plt.axis('equal')
plt.legend(loc=0)
plt.grid(True)
plt.show()